In [1]:
import numpy as np
import pandas as pd
import string



In [2]:
#Task 1 - Create a funtion returning the highest product of three elements.

# Assumed I was supposed to implement without pythons built-in sort()
# with sort() it could be done in 2 lines.

def insertion_sort(array):
    for i in range(1,len(array)):
        value = array[i]
        
        prev_idx = i-1
        while(prev_idx >=0 and value < array[prev_idx]):
            array[prev_idx+1] = array[prev_idx]
            prev_idx -= 1
        array[prev_idx+1] = value
    return array

    
def high_prod(array):
    arr = np.array(insertion_sort(array))
    return arr[-1]*arr[-2]*arr[-3] #The last three elements are then multiplied and returned.

high_prod([1,10,2,6,5,3]) #An example using the high_prod(list) function


300

In [5]:
#Task 3 - Utilize historical data from Oslo City bikes for data analysis and data visualization
#-*- coding: utf-8 -*-

# It takes about a minute to run this cell. Be patient...
try:
    import folium
except:
    !pip install folium
    import folium
try: 
    import seaborn as sns
except: 
    !pip install seaborn
    import seaborn as sns
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#Concentrating on data from the summer of 20 to limit data usage. 
try:
    june = pd.read_csv('06.csv')
    july = pd.read_csv('07.csv')
    august = pd.read_csv('08.csv')
except:
    !wget "data.urbansharing.com/oslobysykkel.no/trips/v1/2020/06.csv"
    !wget "data.urbansharing.com/oslobysykkel.no/trips/v1/2020/07.csv"
    !wget "data.urbansharing.com/oslobysykkel.no/trips/v1/2020/08.csv"
    june = pd.read_csv('06.csv')
    july = pd.read_csv('07.csv')
    august = pd.read_csv('08.csv')
    


stations = pd.concat([july],axis = 0).drop_duplicates(subset=['start_station_name'])
stations.head()


rides_started = {}
rides_ended = {}

all_rides = pd.concat([july], axis=0)

for index,row in all_rides.iterrows():
    if row['start_station_name'] in rides_started:
        rides_started[row['start_station_name']] += 1
    else:
        rides_started[row['start_station_name']] = 1
    if row['end_station_name'] in rides_ended:
        rides_ended[row['end_station_name']] += 1
    else:
        rides_ended[row['end_station_name']]  = 1
    





m = folium.Map([59.911491, 10.757933], zoom_start=11) #Init with logitude and latitude of Oslo
station_dict = {}
#Extracting longitude, latitude and station name and visualizing each station with a circle and name
for index, row in stations.iterrows():
    station_dict[row['start_station_name']] = row['start_station_latitude'], row['start_station_longitude']
    folium.CircleMarker([row['start_station_latitude'], row['start_station_longitude']],
                        radius=15,
                        popup=folium.Popup("<p>"+'<strong>'+str(str(row['start_station_name'].encode('raw_unicode_escape')))[2:-1]+'</strong>' + "</p>"
                              "<p>"+"<em>"+"Turer startet: "+"</em>" + '<strong>'+str(rides_started[row['start_station_name']])+'</strong>' + "</p>"
                              "<p>"+"<em>"+"Turer endt: "+"</em>" + '<strong>'+str(rides_ended[row['start_station_name']])+'</strong>'+"</p>"),
                        fill_color="#e4a73d" # divvy color
                       ).add_to(m)

    
# We create a np.array of the stations containing their respective latitude and logitude
station_array = stations[['start_station_latitude', 'start_station_longitude']]


# plot heatmap to visualize the denisty of the stations
m.add_child(plugins.HeatMap(station_array, radius=40))
m



In [4]:
# The map above shows a map with a integrated heat map describing the density of the stations. By clicking on the differnt
# stations one can also see information about how many rides were started at the give station. Now based on the heat map we will
# invetigate the 10 most used start and end station is situated in a area with high density or not. By looking at this we can 
# conclude whether the areas with high activity satisfies the demand for bicycles and parking.

sort_started_rides = sorted(rides_started.items(), key=lambda x: x[1], reverse=True)
sort_ended_rides = sorted(rides_ended.items(), key=lambda x: x[1], reverse=True)

n = folium.Map([59.911491, 10.757933], zoom_start=13) #Init with logitude and latitude of Oslo

for i in range(10):
   
    folium.CircleMarker([station_dict[sort_started_rides[i][0]][0], station_dict[sort_started_rides[i][0]][1]],
                        radius=0.005*sort_started_rides[i][1],
                        popup=folium.Popup("<p>"+'<strong>'+str(str(sort_started_rides[i][0].encode('raw_unicode_escape')))[2:-1]+'</strong>' + "</p>"
                              "<p>"+"<em>"+"Turer startet: "+"</em>" + '<strong>'+str(sort_started_rides[i][1])+'</strong>'+"</p>"),
                        fill_color="#046304", fill_opacity=0.9 # divvy color
                        
                       ).add_to(n)
    folium.CircleMarker([station_dict[sort_ended_rides[i][0]][0], station_dict[sort_ended_rides[i][0]][1]],
                        radius=0.005*sort_ended_rides[i][1],
                        popup=folium.Popup("<p>"+'<strong>'+str(str(sort_ended_rides[i][0].encode('raw_unicode_escape')))[2:-1]+'</strong>' + "</p>"
                              "<p>"+"<em>"+"Turer startet: "+"</em>" + '<strong>'+str(sort_ended_rides[i][1])+'</strong>'+"</p>"),
                        fill_color="#630404", fill_opacity=0.9 # divvy color
                        
                       ).add_to(n)
    
n.add_child(plugins.HeatMap(station_array, radius=21))
n

# In the mab below the major start stations are marked in dark green with a circle size according number of started rides,
# and major end stations in dark red with size according to number of ended rides. 

# Note that this is only the ten most used start and end stations, and the conclusions drawn here does not necessarily reflect
# on the Oslo bysykkel program as a whole. The reason for this redusction is because it is time comsuming to process that many data points. 

# From the map presented below one can observe that some of the major start stations are located in areas with high density of 
# stations. This is good, as there clearly is a high demand for bicycles there. But the rest of the start stations are located 
# in somewhat less denser areas. An explanation for this could be that people from the outer-city area wants to get to the 
# city-center. This theory is also supported by the fact that none of the major end stations are located in those areas. 

# The two biggest end stations are not located in the most dense areas. This could be an indication of high demand for new 
# stations around those end stations

